# Test that file handler can download things and use the downloaded copy before downloading again

In [1]:
import sys
sys.path.append('../')

import helpers.fileHandler as fh

In [ ]:
#something that works
fh.awsDownloader('ACCESS-CM2', 'ps_Amon', 'historical')

In [ ]:
#something that doesnt work?
fh.awsDownloader('ACCESS-CM2', 'psl_Amon', 'historical')

In [ ]:
fh.loadModelData('ACCESS-CM2', 'psl_Amon', 'ssp126')

In [ ]:
fh.loadModelData('ACCESS-CM2', 'ps_Amon', 'historical')

In [ ]:
fh.awsDownloader('ACCESS-CM2', 'areacella_fx', 'ssp370')

In [ ]:
'assp370'.find('ssp')

In [2]:
fh.esgfDownloader('ACCESS-CM2', 'psl_Amon', 'ssp126', 'r1i1p1f1')

CMIP6.ScenarioMIP.CSIRO-ARCCSS.ACCESS-CM2.ssp126.r1i1p1f1.Amon.psl.gn.v20191108|esgf3.dkrz.de downloading
b'********************************************************************************\n*                                                                              *\n* Note that new functionality to allow authentication without the need for     *\n* certificates is available with this version of the wget script.  To enable,  *\n* use the "-H" option and enter your OpenID and password when prompted:        *\n*                                                                              *\n* $ ACCESS-CM2psl_Amonssp126dl.sh -H [options...]                                     *\n*                                                                              *\n* For a full description of the available options use the help option:         *\n*                                                                              *\n* $ ACCESS-CM2psl_Amonssp126dl.sh -h                             

KeyboardInterrupt: 

In [8]:
from pyesgf.search import SearchConnection

conn = SearchConnection('https://esgf-node.llnl.gov/esg-search')
#conn = SearchConnection('https://esgf-data.dkrz.de/esg-search')
#conn = SearchConnection('https://esgf.nci.org.au/esg-search')

ctx = conn.new_context(
    project='CMIP6', 
    source_id='UKESM1-0-LL', 
    experiment_id='historical', 
    variable='tas', 
    frequency='mon', 
    variant_label='r1i1p1f2',
   #data_node='esgf-data3.ceda.ac.uk'
)
ctx.hit_count

result = ctx.search()
result[0].dataset_id

'CMIP6.CMIP.MOHC.UKESM1-0-LL.historical.r1i1p1f2.Amon.tas.gn.v20190406|aims3.llnl.gov'

In [9]:

result[0].urls

defaultdict(list,
            {'THREDDS': [('http://aims3.llnl.gov/thredds/catalog/esgcet/278/CMIP6.CMIP.MOHC.UKESM1-0-LL.historical.r1i1p1f2.Amon.tas.gn.v20190406.xml#CMIP6.CMIP.MOHC.UKESM1-0-LL.historical.r1i1p1f2.Amon.tas.gn.v20190406',
               'application/xml+thredds')]})

In [13]:
%%writefile dl.sh
result[0].file_context().get_download_script

Overwriting dl.sh


In [31]:
import subprocess
subprocess.check_output('bash dl.sh')

CalledProcessError: Command 'bash dl.sh' returned non-zero exit status 127.

In [ ]:
from pyesgf.logon import LogonManager
lm = LogonManager()
lm.logoff()
lm.is_logged_on()

In [ ]:
myproxy_host = 'esgf-data.dkrz.de'
lm.logon(username=None, password=None, hostname=myproxy_host)
lm.is_logged_on()